In [1]:
from openai import OpenAI
import os

In [2]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [3]:
JOB_DESCRIPTION = """Job description of advertised OpenAI Technical Expert / Data Scientist position:
                TASKS:
                Development and implementation of Machine Learning / Artificial Intelligence models for applications, e.g. in the area of Natural Language Processing or Computer Vision, with a focus on OpenAI technologies to improve our services and processes
                Collaborate closely with other Data Scientists, subject matter experts, and external service providers
                Use case-based evaluation and consulting on the use of analytical methods and approaches (especially OpenAI technologies) to improve our processes and services
                Collaborate closely with our business units as an internal development partner, supporting them from the ideation phase to implementation
                QUALIFICATIONS:
                Master in computer science, mathematics, physics or a comparable qualification with a focus on Machine Learning / AI.
                At least three years of experience in implementing Machine Learning / AI models, including the use of OpenAI technologies
                In-depth expertise in the area of current Machine Learning / AI methods, especially Large Language Models
                Comprehensive knowledge of relevant programming languages, including Python
                Experience with cloud computing platforms, preferably Azure
                Knowledge of natural language processing and/or computer vision
                Strong analytical skills and the ability to solve complex problems
                Strong English communication skills are required, and basic knowledge of German is preferred.
                """

In [27]:
EVALUATION_PROMPT = f"""Job description: {JOB_DESCRIPTION}.
            As an AI interview assistant, your task is to evaluate the quality and depth of the candidate's responses and make a clear hiring decision for this job position.
            Consider the following:
            Does the candidate provide detailed answers that demonstrate their understanding and expertise?
            Can you find tangible examples in their responses that relate to the job description?
            Does the candidate elaborate on how they have used the necessary skills or experiences to overcome challenges or achieve results?
            Do the responses suggest the candidate has the ability to perform well in the role's complexities and challenges?
            If the candidate's responses are inadequate, vague, or don't clearly demonstrate the needed skills or experiences, they may not be a suitable match for the role. In such cases, tactfully communicate this by saying: "Thank you for your responses. However, based on the answers provided, it appears there may be a misalignment with the requirements of the role we're seeking to fill. At this time, we cannot extend an offer. We appreciate your time and effort and wish you the best in your future endeavors."
            If the responses indicate a strong fit for the role, then acknowledge the candidate's suitability by saying: "Thank you for your thoughtful responses. Based on your answers, it appears that your skills, experience, and understanding align well with the requirements of the role. We will be in touch with the next steps."
            """ 

In [56]:
def get_response_from_model(job_description=JOB_DESCRIPTION, nmb_of_questions=3):
    # Generate interview questions based on the job description
    prompt = f"Generate {nmb_of_questions} interview questions for a candidate applying for {job_description}."
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            *[{"role": "user", "content": f"Question {i}:"} for i in range(nmb_of_questions)]
        ]
    )
    return response
 
def parse_model_response(response):  # TODO: check type openai.types.chat.chat_completion.ChatCompletion
    return [question for question in response.choices[0].message.content.split("\n") if question.strip()]  # TODO: use regex
    

def conduct_interview(questions, answers=[]):
    # Conduct the interview
    print("Welcome to the interview!")
    print("Please answer the following questions with a maximum of 20 words each:")
    for i, question in enumerate(questions, 1):
        print(f"Question {i}: {question}")
        answer = input("Your answer: ")
        answers.append(answer)
        print()
    print("Thank you for your answers. We will now evaluate your suitability for the position.")
    return answers

def make_hiring_decision(answers):
    # Make the hiring decision
    prompt = EVALUATION_PROMPT
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": " ".join(answers)}
        ],
        presence_penalty=0.2,
        temperature=0.1
    )
    return response

In [52]:
response = get_response_from_model()
questions = parse_model_response(response)
questions

['1. Can you provide an example of a project where you implemented Machine Learning / AI models using OpenAI technologies? What challenges did you face, and how did you overcome them?',
 '2. How do you stay updated on the latest advancements in Machine Learning / AI, particularly in the realm of Large Language Models? Can you share any specific resources or communities you rely on for continuous learning in this field?',
 '3. Can you walk us through a time when you collaborated with business units as an internal development partner from ideation to implementation of a project? How did you ensure alignment between technical capabilities and business objectives in this process?']

In [58]:
candidate_answers = conduct_interview(questions)

Welcome to the interview!
Please answer the following questions with a maximum of 20 words each:
Question 1: 1. Can you provide an example of a project where you implemented Machine Learning / AI models using OpenAI technologies? What challenges did you face, and how did you overcome them?


Your answer:  In my previous role at [Company], I led a project focused on enhancing customer support efficiency using machine learning and OpenAI technologies. No challenges I had.



Question 2: 2. How do you stay updated on the latest advancements in Machine Learning / AI, particularly in the realm of Large Language Models? Can you share any specific resources or communities you rely on for continuous learning in this field?


Your answer:  If I have time, I read Medium blogs.



Question 3: 3. Can you walk us through a time when you collaborated with business units as an internal development partner from ideation to implementation of a project? How did you ensure alignment between technical capabilities and business objectives in this process?


Your answer:  I hate people. 



Thank you for your answers. We will now evaluate your suitability for the position.


In [59]:
result = make_hiring_decision(candidate_answers)
result

ChatCompletion(id='chatcmpl-9N4kSbb49HeG9qOnDED2RaqeAXx2V', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Thank you for your responses. However, based on the answers provided, it appears there may be a misalignment with the requirements of the role we're seeking to fill. At this time, we cannot extend an offer. We appreciate your time and effort and wish you the best in your future endeavors.", role='assistant', function_call=None, tool_calls=None))], created=1715286824, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=60, prompt_tokens=599, total_tokens=659))